# Nvidia Nemo
In this code we run Nvidia Nemo on all of the dataset

In [1]:
import IPython
import matplotlib.pyplot as plt
import numpy as np
import librosa
from nemo.collections.asr.parts.utils.speaker_utils import (
    rttm_to_labels,
    labels_to_pyannote_object,
)
import os
import IPython
import matplotlib.pyplot as plt
import numpy as np
import librosa
from tqdm import tqdm

from omegaconf import OmegaConf
from nemo.collections.asr.models import ClusteringDiarizer


import wget
import json

[NeMo W 2024-05-30 13:21:15 nemo_logging:349] c:\Users\rakin\anaconda3\envs\nemo\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


In [2]:
# Directories
data_directory = "../Dataset/Audio/Dev"
rttm_directory = "../Dataset/RTTMs/Dev"
sr = 16000
results_directory = "../Results/Oracle_vad"
model_config = "../Config/"
Metadata_test = "../Metadata_test/"

# Loading the pre-trained model
MODEL_CONFIG = os.path.join(model_config, "diar_infer_telephonic.yaml")
if not os.path.exists(MODEL_CONFIG):
    print("Did not find the model config file, downloading it now")
    config_url = "https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/diar_infer_telephonic.yaml"
    MODEL_CONFIG = wget.download(config_url, model_config)
config = OmegaConf.load(MODEL_CONFIG)
pretrained_speaker_model = "titanet_large"
config.diarizer.speaker_embeddings.model_path = pretrained_speaker_model
config.diarizer.speaker_embeddings.parameters.window_length_in_sec = [
    1.5,
    1.25,
    1.0,
    0.75,
    0.5,
]
config.diarizer.speaker_embeddings.parameters.shift_length_in_sec = [
    0.75,
    0.625,
    0.5,
    0.375,
    0.1,
]
config.diarizer.speaker_embeddings.parameters.multiscale_weights = [1, 1, 1, 1, 1]
config.diarizer.oracle_vad = True  # ----> ORACLE VAD
config.diarizer.clustering.parameters.oracle_num_speakers = False
config.device = "cuda"
config.diarizer.out_dir = results_directory  # Set out_dir correctly
config.diarizer.output_dir = results_directory


# Iterate over all files in the directory
for filename in tqdm(os.listdir(data_directory)):
    if filename.endswith(".wav"):
        # Get the file paths
        audio_path = os.path.join(data_directory, filename)
        # Get the RTTM file path. They are are ../Dataset/RTTMs/Dev
        rttm_path = os.path.join(rttm_directory, filename.replace(".wav", ".rttm"))
        signal, sr = librosa.load(audio_path, sr=sr)

        # Get the labels from the RTTM file
        labels = rttm_to_labels(rttm_path)
        reference = labels_to_pyannote_object(labels)

        # Create the metadata dictionary
        meta = {
            "audio_filepath": audio_path,
            "offset": 0,
            "duration": None,
            "label": "infer",
            "text": "-",
            "num_speakers": None,
            "rttm_filepath": rttm_path,
            "uem_filepath": None,
        }

        # Write the metadata to a JSON file
        manifest_filename = filename.replace(".wav", ".json")
        manifest_path = os.path.join(Metadata_test, manifest_filename)
        with open(manifest_path, "w") as fp:
            json.dump(meta, fp)
            fp.write("\n")

        config.diarizer.manifest_filepath = (
            manifest_path  # Use the correct manifest path
        )
        oracle_vad_clusdiar_model = ClusteringDiarizer(cfg=config)
        oracle_vad_clusdiar_model.diarize()

        # Get the RTTMS
        rttm_pred_path = os.path.join(results_directory, "pred_rttms", filename.replace(".wav", ".rttm"))
        pred_labels_neural = rttm_to_labels(rttm_pred_path)

  0%|          | 0/216 [00:00<?, ?it/s]

[NeMo I 2024-05-30 13:21:18 clustering_diarizer:157] Loading pretrained titanet_large model from NGC
[NeMo I 2024-05-30 13:21:18 cloud:58] Found existing object C:\Users\rakin\.cache\torch\NeMo\NeMo_2.0.0rc1\titanet-l\11ba0924fdf87c049e339adbf6899d48\titanet-l.nemo.
[NeMo I 2024-05-30 13:21:18 cloud:64] Re-using file from: C:\Users\rakin\.cache\torch\NeMo\NeMo_2.0.0rc1\titanet-l\11ba0924fdf87c049e339adbf6899d48\titanet-l.nemo
[NeMo I 2024-05-30 13:21:18 common:826] Instantiating model from pre-trained checkpoint


[NeMo W 2024-05-30 13:21:18 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2024-05-30 13:21:18 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2024-05-30 13:21:18 features:305] PADDING: 16
[NeMo I 2024-05-30 13:21:19 save_restore_connector:263] Model EncDecSpeakerLabelModel was successfully restored from C:\Users\rakin\.cache\torch\NeMo\NeMo_2.0.0rc1\titanet-l\11ba0924fdf87c049e339adbf6899d48\titanet-l.nemo.
[NeMo I 2024-05-30 13:21:19 speaker_utils:93] Number of files to diarize: 1
[NeMo I 2024-05-30 13:21:19 clustering_diarizer:287] Subsegmentation for embedding extraction: scale0, ../Results/Oracle_vad\speaker_outputs\subsegments_scale0.json
[NeMo I 2024-05-30 13:21:19 clustering_diarizer:343] Extracting embeddings for Diarization
[NeMo I 2024-05-30 13:21:19 collections:732] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-05-30 13:21:19 collections:733] Dataset loaded with 82 items, total duration of  0.03 hours.
[NeMo I 2024-05-30 13:21:19 collections:735] # 82 files loaded accounting to # 1 labels
[NeMo I 2024-05-30 13:21:20 clustering_diarizer:389] Saved embedding files to ../Results/Oracle

[NeMo W 2024-05-30 13:21:21 nemo_logging:349] c:\Users\rakin\anaconda3\envs\nemo\lib\site-packages\pyannote\metrics\utils.py:200: UserWarning: 'uem' was approximated by the union of 'reference' and 'hypothesis' extents.
      warnings.warn(
    


...

[NeMo I 2024-05-30 13:21:25 features:305] PADDING: 16
[NeMo I 2024-05-30 13:21:26 save_restore_connector:263] Model EncDecSpeakerLabelModel was successfully restored from C:\Users\rakin\.cache\torch\NeMo\NeMo_2.0.0rc1\titanet-l\11ba0924fdf87c049e339adbf6899d48\titanet-l.nemo.


[NeMo W 2024-05-30 13:21:26 clustering_diarizer:411] Deleting previous clustering diarizer outputs.


[NeMo I 2024-05-30 13:21:26 speaker_utils:93] Number of files to diarize: 1
[NeMo I 2024-05-30 13:21:26 clustering_diarizer:287] Subsegmentation for embedding extraction: scale0, ../Results/Oracle_vad\speaker_outputs\subsegments_scale0.json
[NeMo I 2024-05-30 13:21:26 clustering_diarizer:343] Extracting embeddings for Diarization
[NeMo I 2024-05-30 13:21:26 collections:732] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-05-30 13:21:26 collections:733] Dataset loaded with 875 items, total duration of  0.36 hours.
[NeMo I 2024-05-30 13:21:26 collections:735] # 875 files loaded accounting to # 1 labels
[NeMo I 2024-05-30 13:21:26 clustering_diarizer:389] Saved embedding files to ../Results/Oracle_vad\speaker_outputs\embeddings
[NeMo I 2024-05-30 13:21:26 clustering_diarizer:287] Subsegmentation for embedding extraction: scale1, ../Results/Oracle_vad\speaker_outputs\subsegments_scale1.json
[NeMo I 2024-05-30 13:21:26 clustering_diarizer:343] Extracting embeddings for

  1%|          | 2/216 [00:12<22:49,  6.40s/it]


KeyboardInterrupt: 